In [1]:
!python --version
!nvcc --version
!pip install nvcc4jupyter

Python 3.11.11
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


Day 01 -

Source: https://youtu.be/xwbD6fL5qC8?si=AMnLfVoWYBRHiN3y

1. Kernel -> Several threads
2. Grids -> Blocks -> Threads :: GPU -> SM -> SP



In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpini9yd5e".


In [4]:
%%cuda
#include <stdio.h>
__global__ void hello(){
  printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
  hello<<<2, 2>>>();
  cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1



Kernel basics
1. Allocate memory on host
2. Copy from host to device
3. Launch kernel
4. Copy from device to host
5. Free resources

In [24]:
%%cuda
#include <iostream>

__global__ void vecAdd(const float* A, const float* B, float* C, int N) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N){
        C[idx] = A[idx] + B[idx];
    }


}




int main(){

    const int N = 1024;
    const int size = N * sizeof(float);

    float A[N], B[N], C[N];
    float *d_a, *d_b, *d_c;

    // Allocate memory on host
    cudaMalloc(&d_a,  size);
    cudaMalloc(&d_b,  size);
    cudaMalloc(&d_c,  size);

    // Copy from host to device
    cudaMemcpy(d_a, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, B, size, cudaMemcpyHostToDevice);

    // Load vecAdd kernel
    int blocksize = 256;
    int gridsize = ceil(N/blocksize);
    vecAdd <<<gridsize, blocksize>>>(d_a, d_b, d_c, N);

    // Copy from device to host
    cudaMemcpy(C, d_c, size, cudaMemcpyDeviceToHost);

    for(int i = N-10; i<N; i++){
        std::cout << "C[" << i << "] = " << C[i] << std::endl;
    }

    // Free resources
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);



}

C[1014] = 0
C[1015] = 0
C[1016] = 6.83264e-35
C[1017] = 6.74305e-42
C[1018] = 6.86667e-35
C[1019] = 6.74305e-42
C[1020] = 6.72623e-44
C[1021] = 0
C[1022] = 6.8607e-35
C[1023] = 6.74305e-42

